### Importing Libraries

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import  GradScaler
from torch.amp import autocast
import string
import time 
import numpy as np 

In [29]:
import torch
import torch._utils

if not hasattr(torch, '_utils'):
    torch._utils = torch.utils._utils

if not hasattr(torch, '_utils_internal'):
    torch._utils_internal = torch._utils

In [30]:
torch.backends.cudnn.benchmark = True

In [31]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {DEVICE} device")

Using cuda device


### Data Preprocessing & Utilities 

In [32]:
class PassUtils:
    def __init__(self , seq_len = 14):
        self.seq_len = seq_len
        self.charset = string.printable[:-5]
        self.char_to_int = {char: i for i , char in enumerate(self.charset)}
        self.int_to_char = {i: char for i , char in enumerate(self.charset)}
        self.vocab_size = len(self.charset)

    def encode(self , password):
        one_hot = torch.zeros(( self.vocab_size, self.seq_len))
        for i , char in enumerate(password[:self.seq_len]):
            if char in self.char_to_int:
                one_hot[self.char_to_int[char],i] = 1.0
        return one_hot

    def decode(self , tensor):
        if len(tensor.shape) == 2:
            indices = torch.argmax(tensor , dim =0 )
        else:
            indices = torch.argmax(tensor , dim =1 )
        return "".join([self.int_to_char[idx.item()] for idx in indices]).strip()

In [33]:
class Dataset(Dataset):
    def __init__(self , file_path , utils, limit = 1000000):
        self.utils = utils 
        self.passwords = []
        with open(file_path , "r" , encoding="utf-8", errors = "ignore") as f:
            #self.passwords = [line.strip() for line in f if 0 < len(line.strip()) <= self.utils.seq_len]
            for line in f:
                p = line.strip()
                if 0< len(p) <= self.utils.seq_len:
                    self.passwords.append(p)
                if len(self.passwords) >= limit:
                    break
        print(f"Loaded {len(self.passwords)} passwords from {file_path}")
    def __len__(self):
        return len(self.passwords)
    

    def __getitem__(self , idx):
        return self.utils.encode(self.passwords[idx])



### Model Architectue 

In [34]:
class ResBlock(nn.Module):
    def __init__(self , dim):
        super().__init__()
        self.res = nn.Sequential(
            nn.ReLU(True),
            nn.Conv1d(in_channels = dim , out_channels = dim , kernel_size = 3 , padding = 1 ),
            nn.ReLU(True),
            nn.Conv1d(in_channels = dim , out_channels = dim , kernel_size = 3 , padding = 1)
        )

    def forward(self , x):
        return x + ( 0.3 * self.res(x))

In [35]:
class Generator(nn.Module):
    def __init__(self , seq_len , vocab_size , hid_dim = 512):
        super().__init__()
        self.fc = nn.Linear(128 , hid_dim * seq_len)
        self.stack = nn.Sequential( *[ResBlock(hid_dim) for _ in range(5)])
        self.Conv_out = nn.Conv1d(in_channels = hid_dim , out_channels = vocab_size , kernel_size = 1)
        self.seq_len , self.hid_dim  = seq_len , hid_dim
    def forward(self , z):
        out = self.fc(z).view( -1 , self.hid_dim , self.seq_len)
        out = self.stack(out)
        return F.softmax(self.Conv_out(out) , dim = 1)

In [36]:
class Discriminator(nn.Module):
    def __init__(self, seq_len , vocab_size , hid_dim = 512):
        super().__init__()
        self.conv_in =  nn.Conv1d(in_channels = vocab_size , out_channels = hid_dim , kernel_size = 1)
        self.stack = nn.Sequential( *[ResBlock(hid_dim) for _ in range(5)])
        self.fc = nn.Linear(hid_dim * seq_len , 1)
    

    def forward(self , x):
        out = self.conv_in(x)
        out = self.stack(out)
        return self.fc(out.view(out.size(0) , -1))


### Training and Penalties

In [37]:
def GP(D , real , fake , device=DEVICE):
    batch_size = real.size(0)
    alpha = torch.rand(batch_size , 1 ,1).to(device)
    interpolates = (alpha * real + (1 - alpha) * fake).requires_grad_(True)
    d_interpolates = D(interpolates)
    grads = torch.autograd.grad(d_interpolates , interpolates , grad_outputs = torch.ones_like(d_interpolates) , create_graph = True)[0]
    return ((grads.view(grads.size(0) , -1).norm(2 , dim = 1) - 1) ** 2).mean()



In [38]:
def train(epochs=10, batch_size=64, device=DEVICE):
    utils = PassUtils(seq_len=10)
    dataset = Dataset("rockyou-train.txt", utils) 
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)

    G = Generator(10,utils.vocab_size).to(device)
    D = Discriminator(10, utils.vocab_size).to(device)
    
    optim_G = torch.optim.Adam(G.parameters(), lr=1e-4, betas=(0.5, 0.9))
    optim_D = torch.optim.Adam(D.parameters(), lr=1e-4, betas=(0.5, 0.9))
    
    scaler = torch.amp.GradScaler('cuda')

    print(f"Starting training on {len(dataset)} passwords...")

    for epoch in range(epochs):
        start_time = time.time()
        G.train()
        D.train()
        
        for i, real in enumerate(dataloader):
            real = real.float().to(device)
            current_batch_size = real.size(0)
            
    
            optim_D.zero_grad()
            z = torch.randn(current_batch_size, 128).to(device)
            
            with torch.amp.autocast('cuda'):
                fake = G(z)
                d_real = D(real)
                d_fake = D(fake.detach())
                gp_loss = GP(D, real, fake.detach(), device)
    
                d_loss = -torch.mean(d_real) + torch.mean(d_fake) + 10 * gp_loss

            scaler.scale(d_loss).backward()
            scaler.step(optim_D)
    
            if i % 5 == 0:
                optim_G.zero_grad()
                with torch.amp.autocast('cuda'):
    
                    gen_fake = G(z)
                    d_gen_fake = D(gen_fake)
                    g_loss = -torch.mean(d_gen_fake)

                scaler.scale(g_loss).backward()
                scaler.step(optim_G)
            scaler.update() 

    
            if i % 100 == 0:
                
                print(f"Epoch {epoch+1}/{epochs} | Batch {i}/{len(dataloader)} | D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f}", end="\r")

        end_time = time.time()
        
        
        print(f"\n{'='*30}")
        print(f"Epoch [{epoch+1}/{epochs}] Completed!")
        print(f"D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f} | Time: {end_time - start_time:.2f}s")
        
        
        G.eval()
        with torch.no_grad():
            sample_z = torch.randn(1, 128).to(device)
            sample_pass = G(sample_z).squeeze(0)
            print(f"Sample Generated: {utils.decode(sample_pass)}")
        print(f"{'='*30}\n")

    return G, D, utils

### Testing

In [39]:
def crack_meter(password, G, D, utils):
    device = next(D.parameters()).device
    D.eval()
    
    target_tensor = utils.encode(password).to(device).unsqueeze(0)
    with torch.no_grad():
        raw_score = D(target_tensor).item()
        prob = torch.sigmoid(torch.tensor(raw_score)).item()

    charset_size = utils.vocab_size
    theoretical_combinations = charset_size ** len(password)
    
    guesses_needed = theoretical_combinations * (1 - prob + 1e-9)
    gpu_speed = 1e11 
    seconds = guesses_needed / gpu_speed

    print(f"\n[ RESULTS FOR: {password} ]")
    print(f"AI 'Likelihood' Score: {prob:.4f}")
    
    if seconds < 1:
        print("Estimated Crack Time: < 1 Second (AI found a common pattern)")
    elif seconds < 3600:
        print(f"Estimated Crack Time: ~{seconds/60:.2f} Minutes")
    else:
        print(f"Estimated Crack Time: ~{seconds/86400:.2f} Days")

### _____________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________

In [41]:
gen, disc, utils_obj = train(epochs=10, batch_size=64)


Loaded 1000000 passwords from rockyou-train.txt
Starting training on 1000000 passwords...
Epoch 1/10 | Batch 15600/15625 | D Loss: -1.2574 | G Loss: 0.6699
Epoch [1/10] Completed!
D Loss: -1.1850 | G Loss: 0.8359 | Time: 495.67s
Sample Generated: 994459|X9I

Epoch 2/10 | Batch 15600/15625 | D Loss: -1.2691 | G Loss: 0.7749
Epoch [2/10] Completed!
D Loss: -1.1473 | G Loss: 0.7998 | Time: 486.37s
Sample Generated: 02027227]^

Epoch 3/10 | Batch 15600/15625 | D Loss: -0.8296 | G Loss: 0.5049
Epoch [3/10] Completed!
D Loss: -1.0702 | G Loss: 0.5527 | Time: 458.76s
Sample Generated: yelveaK|V"

Epoch 4/10 | Batch 15600/15625 | D Loss: -1.1382 | G Loss: 0.5854
Epoch [4/10] Completed!
D Loss: -1.0282 | G Loss: 0.5747 | Time: 457.36s
Sample Generated: capper*.<$

Epoch 5/10 | Batch 15600/15625 | D Loss: -0.9410 | G Loss: 0.8906
Epoch [5/10] Completed!
D Loss: -1.0660 | G Loss: 0.9863 | Time: 457.91s
Sample Generated: berlluiam;

Epoch 6/10 | Batch 15600/15625 | D Loss: -0.9262 | G Loss: 0.6328

In [42]:
crack_meter("#S23xkdP&_", gen, disc, utils_obj)
crack_meter("7?{5hpJ4vQ", gen, disc, utils_obj)


[ RESULTS FOR: #S23xkdP&_ ]
AI 'Likelihood' Score: 0.6007
Estimated Crack Time: ~2766.94 Days

[ RESULTS FOR: 7?{5hpJ4vQ ]
AI 'Likelihood' Score: 0.5628
Estimated Crack Time: ~3029.64 Days
